# Lec3 Reading要求：
* DLwP: C4&5 (完成C4，C5在另一个文件里)
* <font color='red'>(未完成) </font>DL: 9


# DLwP: C4, Fundamental Machine Learning

### Classification & regression glossary

* Sample/input: one data point
* Prediction/output
* Target -- The truth
* Prediction error/loss value: 衡量model's prediction 与target间差距。
* Classes: A set of possible labels to choose from in classification
* Label
* Ground-truth/annotations: All targets for a dataset.
* Binary classification
* Mutliclass classification
* Multilabel classification: 输出可能是多个label。比如图像识别，可能有的图中既有猫又有狗，那么就有两个label.
* Scalar regression: 输出是单个的连续的scalar value.
* Vector regression: 输出是set of continuous values, 比如a continuous vector.
* Mini-batch/batch. 训练中梯度下降批量大小，通常是8～128间，并且是2的power.


### 数据量少，防止overfitting:
* Hold-out: 取一部分training出来做validation
* K-fold: training均分成k份，每次抽一份出来做validation, 剩下的train. 然后平均k次的结果.最后用所有training data 重新train model.
* Iterated K-fold: 数据量特别少的时候，将上面的K-fold进行P次，每次划分前shuffle data. Performance好，expensive. 


---

### 注意：
* Data representativeness: training data应该足够代表population 性质。所以**randomly shuffle**很重要！
* Arrow of time: 如果是预测将来的情况，就不能随意shuffle. 必须保证test set中的数据都在training set时间之后。
* Redundancy in data: training和test中不应该有重复数据，两者应该无关。

## 4.3. Data preprocessing, feature engineering, and feature learning.
### 1. Data Vectorization
NN中所有输入和输出都是浮点类，所以所有的数据都必须编程tensor, 称为**Data Vectorization**.

###  2. Value Normalization。
需要data满足：
* Take small values：通常都在0～1之间.
* Be homogeneous：所有feature roughly in the same range.<br>
 所以常见的处理是x=(x-mean)/std，从而保证mean=0, std=1.
 
### 3. Missing Values
 通常用0来代替missing values. NN 会learn到这些是missing values，可以ignore。
 
 **但是**，如果testing data中有missing values，而training data中没有，就会出现问题（NN不知道这些是missing values）。需要人为在training中制造
 
### 4. Feature Engineering
找到合适的feature representation



## 4.4. Overfitting and underfitting
最好的方式是获取更多的training data。
但这不一定能实现。
可以限制model存储的信息，这种防止overfitting的方式称为**regularization**.

Model's **capacity**: number of learnable parameters in the model.

需要在overfitting和underfitting之间权衡。通常是从小模型开始（更少的layer，更少的units per layer）, 通过validation loss 去调整，找到合适的model.

除了上面这种人为选取simple model的方式，还可以进行weight regularization. 对loss function加上weight cost, 可以抑止大值weights. 常见两种：
* L1 regularization: L1 norm, weight 绝对值。
* L2 regularization: L2 norm, weight 平方。又称为weight decay.

code如下：




In [0]:
from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                      activation='relu', input_shape=(10000,)))
model.add(layers.Dense(1, activation='sigmoid'))

l2(0.001) means every coefficient in the weight matrix of the layer will add 0.001 \* weight_coefficient_value to the total loss of the network. Note that because this penalty is only added at training time, the loss for this network will be much higher at training than at test time.

也可以同时加l1和l2: <br>
regularizers.l1_l2(l1=0.001, l2=0.01)

### Dropout
Dropout是最常用的regularization方法。
training时，随机扔掉某一层中的一部分output features.
所以下一层收到的这些feature对应的值就是0.

dropout rate通常取在0.2～0.5之间。

test 时，这些units不会drop out, 但是输出值会以dropout rate 来scale down, 去balance.因为test时没有dropout, 那么比training时有更多的active units.

<font color='red'>这里讲test中scale有点迷糊，觉得可能有差？</font> 

The **core idea** is that introducing noise in the output values of a layer can break up happenstance patterns that aren’t significant, which the network will start memorizing if no noise is present.



In [0]:
model.add(layers.Dropout(0.5))

The most common ways to prevent overfitting in neural networks:
* Get more training data.
* Reduce the capacity of the network. 
* Add weight regularization.
* Add dropout.

### 调试model,看overfitting边界
* Add layers.
* Make the layers bigger.
* Train for more epochs.<br>
看 training loss and validation loss, 或其它metrics.
若validation的performance下降，说明overfitting.


### Regularizing and tuning hyperparameters.
* Add dropout
* Try different architectures: 加减层
* Add L1/L2 regularization
* 试不同hyperparameters.
* 可以试试feature engineering.

注意对validation 也不能太多次处理，否则也可能overfit to validation data. 

### Chapter summary
* Define the problem at hand and the data on which you’ll train. Collect this data, or annotate it with labels if need be.
* Choose how you’ll measure success on your problem. Which metrics will you monitor on your validation data?
* Determine your evaluation protocol: hold-out validation? K-fold validation? Which portion of the data should you use for validation?
* Develop a first model that does better than a basic baseline: a model with statistical power.
* Develop a model that overfits.
* Regularize your model and tune its hyperparameters, based on performance on the validation data. A lot of machine-learning research tends to focus only on this step—but keep the big picture in mind.